In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
import numpy as np
import pandas as pd
import scipy        
import warnings
import random
import json

### Read and prepare necessary dataframes

In [3]:
assays_df = pd.read_csv('temp/chembl31-labels-assay-index-nondup.csv')
compounds_df = pd.read_csv('temp/chembl31-labels-compound-index.csv')
target_df = pd.read_csv("output/assay_target_map.csv")

In [4]:
# Have to retrieve smiles from another csv
"""
df_with_smiles = pd.read_csv('temp/cellpainting-index.csv')
cp_inchi_smiles_map = pd.merge(compounds_df[["INCHIKEY"]], df_with_smiles[["INCHIKEY", "CPD_SMILES"]], how="left", on="INCHIKEY").drop_duplicates(subset="INCHIKEY")
cp_inchi_smiles_map.to_csv("temp/cp_inchi_smiles_map.csv", index=False)
"""

'\ndf_with_smiles = pd.read_csv(\'temp/cellpainting-index.csv\')\ncp_inchi_smiles_map = pd.merge(compounds_df[["INCHIKEY"]], df_with_smiles[["INCHIKEY", "CPD_SMILES"]], how="left", on="INCHIKEY").drop_duplicates(subset="INCHIKEY")\ncp_inchi_smiles_map.to_csv("temp/cp_inchi_smiles_map.csv", index=False)\n'

In [5]:
label_matrix = (scipy.io.mmread('temp/chembl31-label-matrix.mtx'))
B = label_matrix.todense()
df = pd.DataFrame(data=B, index=list(compounds_df["INCHIKEY"]) ,columns=list(assays_df["ASSAY_ID"]))
df

,737823,737824,688422,688810,688812,688816,688724,688739,688759,688238,...,1301593,1301717,1301709,688671,845206,1301866,1301890,1301859,1301893,1495346
IENZQIKPVFGBNW-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
GSDSWSVVBLHKDQ-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,0
CGIGDMFJXJATDK-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,0
DSXXEELGXBCYNQ-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,0,0,1,0,0,0,0
MYSWGUAQZAJSOK-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YAJYINBQFXCAPI-WENCSYSZSA-N,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
DIBJIDYNEHVBAV-KFRKLSNLSA-N,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
YAJYINBQFXCAPI-VPYPWEPUSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAJYINBQFXCAPI-RZIGYZOXSA-N,0,0,0,-1,0,0,0,0,-1,-1,...,0,0,0,0,0,0,0,0,0,0


### Switch missing data to -1, and negative labels to 0

In [7]:
# Convert 0 to -1, and -1 to 0 
df_switch = df.copy(deep=True)
df_switch[df_switch == 0] = -2
df_switch[df_switch == -1] = 0
df_switch[df_switch == -2] = -1
df_switch

,737823,737824,688422,688810,688812,688816,688724,688739,688759,688238,...,1301593,1301717,1301709,688671,845206,1301866,1301890,1301859,1301893,1495346
IENZQIKPVFGBNW-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1
GSDSWSVVBLHKDQ-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,0,-1,-1,-1,-1,-1,-1
CGIGDMFJXJATDK-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,0,-1,-1,-1,-1,-1,-1
DSXXEELGXBCYNQ-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,1,-1,-1,-1,-1
MYSWGUAQZAJSOK-UHFFFAOYSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YAJYINBQFXCAPI-WENCSYSZSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
DIBJIDYNEHVBAV-KFRKLSNLSA-N,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
YAJYINBQFXCAPI-VPYPWEPUSA-N,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
YAJYINBQFXCAPI-RZIGYZOXSA-N,-1,-1,-1,0,-1,-1,-1,-1,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


### Split test assay

##### Criteria:
 1. No overlaps between targets in the pretrain or test set
 2. Test assays must have >= 96 compounds
 3. Test assays must have active ratio in between 30% to 70%

In [8]:
# Choose test assays that have >= 128 compounds and active ratio between 30% and 70%
test_assays = []
for assay in df_switch.columns:
    v = df_switch.loc[:,assay].value_counts()
    # Not choose if less that 128 datapoints
    if v[0]+v[1] < 96:
        continue
    # Not choose if active ratio not between 30% and 70%
    active_ratio = v[1]/(v[0]+v[1])
    if active_ratio>0.7 or active_ratio<0.3:
        continue
    test_assays.append(assay)

In [9]:
# Eliminate test assay if share target with pretrain assay
# Because we need more test assays, we will not eliminate assays with target CHEMBL612545(UNCHECKED) out of test assays
list_target_train = list(target_df[~target_df["ASSAY_ID"].isin(test_assays)]["target_chembl_id"])
for assay in test_assays:
    target = target_df[target_df["ASSAY_ID"] == assay]["target_chembl_id"].values[0]
    if target in list_target_train and target[0]!="CHEMBL612545":
        test_assays.remove(assay)

In [10]:
# Create train, val and test assays
train_val_assays = [i for i in assays_df["ASSAY_ID"] if i not in test_assays]
val_assays = random.sample(train_val_assays, int(len(train_val_assays)/8))
train_assays = [i for i in train_val_assays if i not in val_assays]
dict_assay = {
    'train':train_assays,
    'val':val_assays,
    'test':test_assays
}
print(f"There are {len(test_assays)} test assays.")
print(f"There are {len(train_assays)} train assays.")
print(f"There are {len(val_assays)} validation assays.")

There are 18 test assays.
There are 161 train assays.
There are 22 validation assays.


In [11]:
# Export JSON file
json_object = json.dumps(dict_assay)
 
# Writing to sample.json
with open("output/data_split.json", "w") as outfile:
    outfile.write(json_object)